In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import Image 
from IPython.core.display import HTML
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print("imported")

imported


In [2]:
!pip install folium
import folium

In [3]:
from geopy.geocoders import Nominatim

In [4]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
site = requests.get(link)

data = pd.read_html(site.content)

df = pd.DataFrame(data[0])
df.rename(columns={"Postal Code":"PostalCode"}, inplace = True)

In [5]:

test= df.loc[df["Borough"]=="Not assigned"]
df.drop(test.index, inplace= True, axis=0)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [55]:
df.loc[df["Neighbourhood"] == "Not assigned"] = df.loc[:,["Borough"]]
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
36,M1G,Scarborough,Woburn,43.695344,-79.318389
45,M1H,Scarborough,Cedarbrae,43.712751,-79.390197


In [56]:
df.groupby("PostalCode")
df.sort_values("PostalCode", ascending=True, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
36,M1G,Scarborough,Woburn,43.695344,-79.318389
45,M1H,Scarborough,Cedarbrae,43.712751,-79.390197


In [58]:
df.shape

(68, 5)

 I used the provided CSV file to get the location data of the neighbour hoods of toronto, then I sorted it ascendingly according to the postal code.

In [8]:
lat_lng_data = pd.read_csv("https://cocl.us/Geospatial_data")
lat_lng_data.sort_values("Postal Code", ascending=True, inplace=True)
lat_lng_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I sorted the neighbourhoods dataframe ascendingly according to the postal code, the I added the latitude and longitude columns to the neighbourhoods dataframe and dropped the NAN values.

In [9]:
df["Latitude"] = lat_lng_data["Latitude"]
df["Longitude"] = lat_lng_data["Longitude"]
df.dropna(inplace=True, axis=0)
df.dropna(inplace=True, axis=0)
df.head(20)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
36,M1G,Scarborough,Woburn,43.695344,-79.318389
45,M1H,Scarborough,Cedarbrae,43.712751,-79.390197
54,M1J,Scarborough,Scarborough Village,43.657162,-79.378937
63,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.711695,-79.416936
72,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.709577,-79.445073
81,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.673185,-79.487262
90,M1N,Scarborough,"Birch Cliff, Cliffside West",43.653654,-79.506944


In [10]:
CLIENT_ID = "WBND0Y2BJEYSLG2RESBF12LUXVSWOQOJFNUBOMTH2WGJO00M"
CLIENT_SECRET = "0QMT33Y0WIM10S4X53PKDAVS2IRD5DWSZWDBWJITRGC0GZ4N"
VERSION = "20200720"
RADIUS = 500
LIMIT = 100

I created this function to create a dataframe of nearby venues to each neighbourhood.

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(df["Neighbourhood"],
                                 df["Latitude"],
                                 df["Longitude"],
                                 RADIUS)

toronto_venues.head(10)


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, 

In [14]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_onehot.shape

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


calculating the average number of venues in each neighbourhood

In [15]:

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped


,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.250000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.029412,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Berczy Park,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Birch Cliff, Cliffside West",0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Caledonia-Fairbanks,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Cedarbrae,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Central Bay Street,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Christie,0.023810,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,...,0.023810,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [42]:
num_top_venues = 5

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create a new dataframe
columns = ['Neighborhood']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop
1,Bayview Village,Mexican Restaurant,Sushi Restaurant,Trail,Jewelry Store,Donut Shop
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Diner,Yoga Studio,Arts & Crafts Store,Café
3,Berczy Park,Grocery Store,Shopping Mall,Bank,Hotel,Park
4,"Birch Cliff, Cliffside West",River,Women's Store,Curling Ice,Donut Shop,Doner Restaurant


Clustered the data using the kmeans algorith and created a new dataframe with the cluster number for each element. I used the dropna() method to make sure there aren't any NAN values in the dataframe, the I changed the type of the Cluster labels column to int to be able to use it as an index.

In [51]:

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged["Cluster Labels"].dropna(inplace=True, axis=0)
toronto_merged["Cluster Labels"].astype(int)


9      0
18     0
27     0
36     0
45     0
54     0
63     0
72     0
81     0
90     2
99     0
46     0
55     0
64     0
73     0
82     0
91     0
100    0
2      0
11     0
20     0
47     0
56     0
65     0
74     0
83     0
92     0
3      0
12     0
21     1
      ..
102    0
4      0
13     0
22     0
31     0
40     3
49     0
58     0
67     0
76     0
85     0
94     4
5      0
14     3
23     3
32     0
41     0
50     3
59     0
68     0
77     0
86     0
95     0
6      0
8      0
17     0
26     0
80     0
89     0
98     3
Name: Cluster Labels, Length: 67, dtype: int64

Used the dataframe to draw the map and superposition the clusters on it.

In [54]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
